# Al-Qur'an

Al-Qur'an adalah kitab suci umat Islam di seluruh dunia yang berfungsi sebagai petunjuk bagi manusia dalam mengarahkan kehidupannya agar mencapai kehidupan yang bahagia di dunia dan di akhirat.

Ditinjau dari segi kebahasaan (etimologi), Al-Qur'an berasal dari bahasa Arab yang berarti "bacaan" atau "sesuatu yang dibaca berulang-ulang". Kata Al-Qur'an adalah bentuk kata benda (masdar) dari kata kerja qara'a yang artinya membaca.

Al-quran menurut terminologi yaitu firman Allah yang mengandung mukjizat yang diturunkan kepada rasul atau Baginda Rasulullah shallallahu alaihi wa sallam nabi Muhammad shallallahu alaihi wasallam dengan perantara malaikat jibril yang tertulis di dalam mushab itu yang disampaikan kepada kita secara mutawatir, yang membacanya dianggap ibadah dimulai dari surat Al-Fatihah dan diakhiri dengan surat An-Nas.

Secara garis besar, Al-Qur'an mengandung ajaran tentang aqidah, syariah, dan akhlak, namun al-Qur'an juga mengandung isyarat-isyarat ilmiah yakni mengandung ayat-ayat sains dan teknologi. Tidak diragukan lagi, ini adalah karya terbaik dalam literatur Arab dan diwahyukan oleh Allah (Tuhan) kepada Rasul-Nya, Nabi Muhammad Shalallahu alayhi wa sallam melalui malaikat Jibril.

Diwahyukan secara berangsur-angsur kepada Nabi Muhammad Shalallahu alayhi wa sallam mulai dari 17 Ramadan 610 Masehi hingga 632 Masehi yang bertepatan dengan wafatnya Nabi Muhammad Shalallahu Alayhi wa sallam. Kitab ini dibagi menjadi 30 juz, 114 Surah dan 6.000+ ayat

# Import Library

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud, STOPWORDS
import re

# Load Dataset & Memahami Data

Import dataset yang berisi ayat-ayat alquran beserta nomor surah dan nomor ayat

In [4]:
quran_english = pd.read_csv('/kaggle/input/the-holy-quran/en.yusufali.csv')

quran_english

,Surah,Ayah,Text
0,1,1,"In the name of Allah, Most Gracious, Most Merc..."
1,1,2,"Praise be to Allah, the Cherisher and Sustaine..."
2,1,3,"Most Gracious, Most Merciful;"
3,1,4,Master of the Day of Judgment.
4,1,5,"Thee do we worship, and Thine aid we seek."
...,...,...,...
6231,114,2,"The King (or Ruler) of Mankind,"
6232,114,3,"The god (or judge) of Mankind,-"
6233,114,4,"From the mischief of the Whisperer (of Evil), ..."
6234,114,5,(The same) who whispers into the hearts of Man...


In [5]:
quran_english.shape

(6236, 3)

NameError: name 'df' is not defined

Dari output di atas menunjukkan:
*   Al-Quran memiliki 6.236 ayat, tidak termasuk Bismillahirrahmanirrahim.
*   6.348 ayat termasuk Bismillahirrahmanirrahim.
*   Al-Quran dibagi menjadi 114 bab, yang disebut surah, dan setiap surah dibagi menjadi sejumlah ayat






In [6]:
quran_english[quran_english['Text'].str.contains('In the name of Allah, Most Gracious, Most Merciful')]


,Surah,Ayah,Text
0,1,1,"In the name of Allah, Most Gracious, Most Merc..."
3188,27,30,"""It is from Solomon, and is (as follows): 'In ..."



Dari hasil di atas dapat diketahui:
*   "Bismillahirrahmanirrahim" berarti "Dengan menyebut nama Allah Yang Maha Pengasih lagi Maha Penyayang"
*   Ada 2 surah yang mengandung kata "Bismillahirrahmanirrahim" yaitu Surah Al-Fatihah ayat 1 dan Surah An-Naml ayat 30



Selanjutnya, dilakukan pengecekan nilai null (kosong) pada dataframe

In [7]:
# Memeriksa value null jika ada untuk memastikan dataset bebas dari kesalahan.
quran_english.isna().sum()

Surah    0
Ayah     0
Text     0
dtype: int64

Dari hasil di atas terlihat tidak ada data yang bernilai null, selanjutnya mengimpor dataset yang berisikan daftar nama-nama surah dalam Al-Qur'an

In [8]:
# Import dataset nama surah
surah_names = pd.read_csv('/kaggle/input/surah-names/surah_names_english.csv', names=['Surah', 'Surah Name'])

# Menghapus whitespace di awal nama
surah_names["Surah Name"] = surah_names["Surah Name"].str[1:]

surah_names

,Surah,Surah Name
0,1,Al-Fatihah (the Opening)
1,2,Al-Baqarah (the Cow)
2,3,Aali Imran (the Family of Imran)
3,4,An-Nisa’ (the Women)
4,5,Al-Ma’idah (the Table)
...,...,...
109,110,An-Nasr (the Help)
110,111,Al-Masad (the Palm Fiber)
111,112,Al-Ikhlas (the Sincerity)
112,113,Al-Falaq (the Daybreak)


Menggabungkan dataframe nama surah dengan dataframe quran pada nomor surah